# Libraries Import

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install mtcnn
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import torch.utils.data as data_utils
import matplotlib.pyplot as plt
import os

from keras.models import Model, Sequential, model_from_json
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from matplotlib.patches import Rectangle 
from mtcnn.mtcnn import MTCNN
from scipy import spatial, optimize, stats, ndimage, misc, 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset

# Couple Images Pre-processing Architecture 
## Extraction of couple faces ---> (Cosine Distance, Euclidean Distance, Pearson Correlation)

In [ ]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [ ]:
model.load_weights('/content/gdrive/MyDrive/A - Academics UM/A - Machine Learning/Grp Project/Final Trial v1.0/vgg_face_weights.h5')
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

### Methods to extract couple faces in image dataset

In [ ]:
def extract_face(filename, image_size=(224, 224)):
  image = cv2.imread(filename)
  image_data = np.asarray(image)
  detector = MTCNN()
  results = detector.detect_faces(image_data)
  x_0, y_0, width_0, height_0 = results[0]['box']
  x_0, y_0 = abs(x_0), abs(y_0)
  x_0_up, y_0_up = x_0 + width_0, y_0 + width_0
  face_0 = image_data[y_0:y_0_up, x_0:x_0_up]
  face_array_0 = cv2.resize(face_0,image_size,interpolation=cv2.INTER_CUBIC)
  face_array_0 = face_array_0.reshape(1, 224, 224, 3)

  x_1, y_1, width_1, height_1 = results[1]['box']
  x_1, y_1 = abs(x_1), abs(y_1)
  x_1_up, y_1_up = x_1 + width_1, y_1 + width_1
  face_1 = image_data[y_1:y_1_up, x_1:x_1_up]
  face_array_1 = cv2.resize(face_1,image_size,interpolation=cv2.INTER_CUBIC)
  face_array_1 = face_array_1.reshape(1, 224, 224, 3)
  return face_array_0, face_array_1

def get_embeddings(file):
  faces_0, faces_1 = extract_face(file)
  samples_0 = np.asarray(faces_0, 'float32')
  samples_0 = preprocess_input(samples_0)
  representation_0 = vgg_face_descriptor.predict(samples_0)
  samples_1 = np.asarray(faces_1, 'float32')
  samples_1 = preprocess_input(samples_1)
  representation_1 = vgg_face_descriptor.predict(samples_1)
  return representation_0, representation_1

def draw_image_with_boxes(filename, result_list):
  data = plt.imread(filename)
  plt.imshow(data)
  ax = plt.gca()
  for result in result_list:
    x, y, width, height = result['box']
    rect = Rectangle((x, y), width, height, fill=False, color='red')
    ax.add_patch(rect)
  plt.show()

def findFace(filename):
  pixels = plt.imread(filename)
  detector = MTCNN()
  faces = detector.detect_faces(pixels)
  draw_image_with_boxes(filename, faces)

def twoFace(filename):
  image = cv2.imread(filename)
  image_data = np.asarray(image)
  detector = MTCNN()
  results = detector.detect_faces(image_data)
  return len(results)

def calculateCompatibility(file):
  representation_0, representation_1 = get_embeddings(file)
  findFace(file)
  d = spatial.distance.cosine(representation_0, representation_1)
  print("Compatibility : {:.5f}%".format(d * 100))


### Methods to compute (Cosine Distance, Euclidean Distance, Pearson Correlation)

In [ ]:
def distanceGenerator(file):
  representation_0, representation_1 = get_embeddings(file)
  #print((representation_0[0]))
  #print((representation_1[0]))
  dCosine = spatial.distance.cosine(representation_0, representation_1)
  dEuc = np.linalg.norm(representation_0 - representation_1)
  #print(type(representation_0))
  Pear = stats.pearsonr(representation_0[0], representation_1[0])
  #print(Pear)
  return dCosine, dEuc, Pear[0], Pear[1]

def datasetFromImage(path, couple):
  dCosineArr = np.array([])
  dEucArr = np.array([])
  PearRArr = np.array([])
  PearPArr = np.array([])
  for image_path in os.listdir(path):
    input_path = os.path.join(path, image_path)
    if twoFace(input_path) == 2:
      dCosine, dEuc, PearR, PearP = distanceGenerator(input_path)
      dCosineArr = np.append(dCosineArr, dCosine)
      dEucArr = np.append(dEucArr, dEuc)
      PearRArr = np.append(PearRArr, PearR)
      PearPArr = np.append(PearPArr, PearP)
  if couple == True:
    LabelArr = np.ones(dCosineArr.shape)
  else:
    LabelArr = np.zeros(dCosineArr.shape)
  return dCosineArr, dEucArr, PearRArr, PearPArr, LabelArr

def dataset(couplePath, notCouplePath):
  dCosineArrC, dEucArrC, PearRArrC, PearPArrC, LabelArrC = datasetFromImage(couplePath, True)
  dCosineArrNC, dEucArrNC, PearRArrNC, PearPArrNC, LabelArrNC = datasetFromImage(notCouplePath, False)
  dCosineArr = np.concatenate([dCosineArrC,dCosineArrNC])
  dEucArr = np.concatenate([dEucArrC,dEucArrNC])
  PearRArr = np.concatenate([PearRArrC,PearRArrNC])
  PearPArr = np.concatenate([PearPArrC,PearPArrNC])
  LabelArr = np.concatenate([LabelArrC,LabelArrNC])
  dataset = pd.DataFrame({'Cosine': dCosineArr, 'Euclidean': dEucArr, 'PearsonR': PearRArr, 'PearsonP': PearPArr, 
                          'Label': LabelArr}, columns=['Cosine', 'Euclidean', 'PearsonR', 'PearsonP', 'Label'])
  return dataset

# Couple Compatibility Model

In [ ]:
seed = 98
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

trainDf = pd.read_csv('/content/gdrive/MyDrive/Final Trial v1.0/dataset.csv')
trainLabel = trainDf.pop('Label')
train_tensor = data_utils.TensorDataset(torch.tensor(trainDf.values.astype(np.float64)), torch.tensor(trainLabel.values.astype(np.float64)))